In [1]:
import libspn as spn
import random
import os
import tensorflow as tf
from graphspn.spn.spn_model import SpnModel
from graphspn.graphs.template import NodeTemplate, ThreeNodeTemplate, ThreeRelTemplate
from graphspn.spn.template_spn import TemplateSpn, NodeTemplateSpn
from graphspn.experiments.scripts.topo_map_dataset import TopoMapDataset
from graphspn.experiments.scripts.topo_map import CategoryManager, ColdDatabaseManager
from graphspn.util import ABS_DIR
import numpy as np

DB_ROOT = os.path.join(ABS_DIR, "experiments", "dataset", "cold-topomaps")
GROUNDTRUTH_ROOT = os.path.join(ABS_DIR, "experiments", "dataset", "cold-groundtruth")

/home/kaiyuzh/repo/deep-semantic-mapping/deepsm/util.py:299: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


In [2]:
def setup_dataset(db,):
    dataset = TopoMapDataset(DB_ROOT)
    dataset.load(db, skip_unknown=True,
                 skip_placeholders=True, single_component=False)
    return dataset

In [3]:
def test_marginal_inference_expand(sess, tspn):
    groundtruth = [2, 2, 2]
    query = [2, -1, 2]
    query_lh = create_likelihoods_vector(groundtruth,
                                         tspn,
                                         (0.5, 0.7), (0.0004, 0.00065),
                                         (0.5, 0.7), (0.17, 0.22),
                                         masked_sample=query)
    marginals = tspn.marginal_inference(sess, np.array(query), query_lh, normalize=True)
    print(marginals)
    most_likely = query
    for i in marginals:
        most_likely[i] = np.argmax(marginals[i])
    print(most_likely)  

In [4]:
def create_likelihoods_for_single_node(catg,
                                       high_likelihood_correct, low_likelihood_correct,
                                       high_likelihood_incorrect, low_likelihood_incorrect, 
                                       masked=False, uniform_for_incorrect=False,
                                       consider_placeholders=False, is_placeholder=False):
    """
    Creates a numpy.array of shape (m,) where m is the number of semantic categories. Each element
    is a float number indicating the likelihood of that category.

    catg (int): true category number (semantic value)
    high_likelihood_(in)correct (tuple): the min & max likelihood of the semantics variable's true class.
    low_likelihood_(in)correct (tuple): the min & max likelihood of the masked semantics variable's true class.
                and the other (untrue) classes.
    masked (bool): True if this semantic value will be masked to -1.
    uniform_for_incorrect (bool): True if use uniform distribution for likelihoods in incorrectly classified nodes.
    consider_placeholders (bool): If True, will set the likelihoods for a placeholder node to be uniform, regardless
                                  of the high/low likelihoods setting.
    is_placeholder (bool): If True, the current node is a placeholder.
    """
    likelihoods = np.zeros((CategoryManager.NUM_CATEGORIES,))
    if not masked:
        for k in range(CategoryManager.NUM_CATEGORIES):
            if catg == k:
                likelihoods[k] = random.uniform(high_likelihood_correct[0], high_likelihood_correct[1])
            else:
                likelihoods[k] = random.uniform(low_likelihood_correct[0], low_likelihood_correct[1])
    else:
        highest = float('-inf')
        # Randomly select a non-groundtruth class to be the correct class
        cc = random.randint(0, CategoryManager.NUM_CATEGORIES)
        while cc == catg:
            cc = random.randint(0, CategoryManager.NUM_CATEGORIES)
        for k in range(CategoryManager.NUM_CATEGORIES):
            if uniform_for_incorrect or (consider_placeholders and is_placeholder):
                likelihoods[k] = 1.0
            else:
                if cc == k:
                    likelihoods[k] = random.uniform(high_likelihood_incorrect[0], high_likelihood_incorrect[1])
                else:
                    likelihoods[k] = random.uniform(low_likelihood_incorrect[0], low_likelihood_incorrect[1])
    # normalize so that likelihoods sum up to 1.
    likelihoods = likelihoods / np.sum(likelihoods)
    return likelihoods



def create_likelihoods_vector(sample, model,
                              high_likelihood_correct, low_likelihood_correct,
                              high_likelihood_incorrect, low_likelihood_incorrect, 
                              masked_sample=None):
    """
    Creates a numpy.array of shape (n, m) where n is the number of semantics variables (per template).
    m is the number of semantic categories. 

    sample (list): contains semantic variable values, in order.
    model (TemplateModel): the spn model that `sample` suits for.
    high_likelihood (tuple): the min & max likelihood of the semantics variable's true class.
    low_likelihood (tuple): the min & max likelihood of the masked semantics variable's true class.
                and the other (untrue) classes.

    masked_sample (list): Masked sample that contains -1. For masked values,
                  we reverse the pattern of assigning likelihoods to classes.
    """
    likelihoods = np.zeros((model.num_nodes, CategoryManager.NUM_CATEGORIES))

    for i in range(len(sample)):
        likelihoods[i] = create_likelihoods_for_single_node(sample[i],
                                                            high_likelihood_correct, low_likelihood_correct,
                                                            high_likelihood_incorrect, low_likelihood_incorrect, 
                                                            masked=masked_sample is not None and masked_sample[i] == -1)

    return likelihoods


# Building SPN

In [5]:
CategoryManager.TYPE = "FULL"
CategoryManager.init()    
dataset = setup_dataset("Stockholm")

In [6]:
sess = tf.Session()

In [7]:
tspn = NodeTemplateSpn(ThreeNodeTemplate, num_vals=CategoryManager.NUM_CATEGORIES)
SpnModel.print_structure(tspn._root, sess)

Generating SPN structure...
Node (Sum)
    Input(PermProducts1, None) (PermProducts)
PermProducts1 (PermProducts)
    Input(Conc_NodeTemplateSpn_3, [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]) (Concat)
    Input(Sums1.1/ParallelSums1.1, None) (ParSums)
Conc_NodeTemplateSpn_3 (Concat)
    Input(Catg_NodeTemplateSpn_3, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]) (IVs)
Sums1.1/ParallelSums1.1 (ParSums)
    Input(PermProducts2, None) (PermProducts)
PermProducts2 (PermProducts)
    Input(Conc_NodeTemplateSpn_3, [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]) (Concat)
    Input(Conc_NodeTemplateSpn_3, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) (Concat)


In [8]:
tspn.generate_random_weights()
tspn.init_weights_ops()
tspn.init_learning_ops()
tspn.initialize_weights(sess)
sess.run(tf.global_variables_initializer())

# train
samples = np.array(dataset.create_template_dataset(ThreeNodeTemplate)['Stockholm'],
                   dtype=np.int32)
train_likelihoods, test_likelihoods \
    = tspn.train(sess, samples, shuffle=True, batch_size=1000,
                 likelihood_thres=0.05, num_epochs=5, dgsm_lh=None,
                 samples_test=None, dgsm_lh_test=None)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/kaiyuzh/repo/libspn/libspn/utils/math.py:466: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Generating weight initialization Ops...
Initializing learning Ops...
Initializing weights...
Resetting accumulators...
Shuffling...
Starts training. Maximum epochs: 5   Likelihood threshold: 0.050
EPOCH 0 BATCH 0 SAMPLES 0 1000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 1 SAMPLES 1000 2000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 2 SAMPLES 2000 3000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 3 SAMPLES 3000 4000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 4 SAMPLES 4000 5000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 5 SAMPLES 5000 6000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 6 SAMPLES 6000 7000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 7 SAMPLES 7000 8000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 8 SAMPLES 8000 9000   prev likelihood 100 likelihood 0
Smoothing:  30.0
EPOCH 0 BATCH 9 SAMPLES 9000 100

Smoothing:  13.479869
EPOCH 4 BATCH 10 SAMPLES 10000 11000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 11 SAMPLES 11000 12000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 12 SAMPLES 12000 13000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 13 SAMPLES 13000 14000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 14 SAMPLES 14000 15000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 15 SAMPLES 15000 16000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 16 SAMPLES 16000 17000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 17 SAMPLES 17000 18000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 18 SAMPLES 18000 19000   prev likelihood -4.627866 likelihood -4.60641
Smoothing:  13.479869
EPOCH 4 BATCH 1

In [9]:
tspn.expand()
SpnModel.print_structure(tspn._root, sess)

Expanding...
Node (Sum)
    Input(PermProducts1, None) (PermProducts)
PermProducts1 (PermProducts)
    Input(Conc_NodeTemplateSpn_3, [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]) (Concat)
    Input(Sums1.1/ParallelSums1.1, None) (ParSums)
Conc_NodeTemplateSpn_3 (Concat)
    Input(Product, None) (Product)
    Input(Product_1, None) (Product)
    Input(Product_2, None) (Product)
    Input(Product_3, None) (Product)
    Input(Product_4, None) (Product)
    Input(Product_5, None) (Product)
    Input(Product_6, None) (Product)
    Input(Product_7, None) (Product)
    Input(Product_8, None) (Product)
    Input(Product_9, None) (Product)
    Input(Product_10, None) (Product)
    Input(Product_11, None) (Product)
    Input(Product_12, None) (Product)
    Input(Product_13, None) (Product)
    Input(Product_14, None) (Product)
    Input(Product_15, None) (Product)
    Input(Product_16, None) (Product)
    Input(Product_17, None) (Product)
    Input(Product_18, None) (Product)
    Input(Product_19, N

In [10]:
spn.display_spn_graph(tspn.root)

In [11]:
test_marginal_inference_expand(sess, tspn)

InvalidArgumentError: You must feed a value for placeholder tensor 'Catg_NodeTemplateSpn_3/Placeholder' with dtype int32 and shape [?,3]
	 [[Node: Catg_NodeTemplateSpn_3/Placeholder = Placeholder[dtype=DT_INT32, shape=[?,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Catg_NodeTemplateSpn_3/Placeholder', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/tor

In [13]:
sess.run(tspn._log_likelihood_root, feed_dict={tspn._catg_inputs: np.array([[1,1,1]])})

array([[-2.801613]], dtype=float32)

In [14]:
sess.run(tspn._log_likelihood_root, feed_dict={tspn._semantic_inputs: np.array([[1,1,1]])})

InvalidArgumentError: You must feed a value for placeholder tensor 'Catg_NodeTemplateSpn_3/Placeholder' with dtype int32 and shape [?,3]
	 [[Node: Catg_NodeTemplateSpn_3/Placeholder = Placeholder[dtype=DT_INT32, shape=[?,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Catg_NodeTemplateSpn_3/Placeholder', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/kaiyuzh/pyenv/py3/lib/python3.5/site-packages/tor

In [15]:
tspn._learning_algorithm

libspn.learning.em.EMLearning

In [20]:
tspn._log_likelihood_root.consumers()

[<tf.Operation 'TrueMPEPath/ones_like/Shape' type=Shape>,
 <tf.Operation 'Mean' type=Mean>]